In [5]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = 'cuda:0'
G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load('autovc.ckpt' ,map_location='cuda:0')
G.load_state_dict(g_checkpoint['model'])

metadata = pickle.load(open('metadata.pkl', "rb"))

spect_vc = []

for sbmt_i in metadata:
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)          

In [6]:
type(spect_vc)

list

In [7]:
spect_vc

[('p225xp225',
  array([[0.38280222, 0.32192808, 0.27241027, ..., 0.18827905, 0.18116055,
          0.1747563 ],
         [0.3774055 , 0.30501315, 0.3142484 , ..., 0.23313922, 0.18586272,
          0.14988673],
         [0.38855177, 0.28357574, 0.2739416 , ..., 0.13206424, 0.10279437,
          0.10369422],
         ...,
         [0.57143515, 0.6357634 , 0.5616601 , ..., 0.21222197, 0.19688365,
          0.182007  ],
         [0.5051674 , 0.50225836, 0.48195556, ..., 0.18236066, 0.16847168,
          0.15469953],
         [0.44893792, 0.44055843, 0.4609493 , ..., 0.14624406, 0.1329227 ,
          0.13138215]], dtype=float32)),
 ('p225xp228',
  array([[0.4372439 , 0.35739934, 0.30414355, ..., 0.20524159, 0.19167477,
          0.17206734],
         [0.44479764, 0.35146436, 0.31340063, ..., 0.23535405, 0.23473123,
          0.22268619],
         [0.4352065 , 0.36368087, 0.34444585, ..., 0.2080973 , 0.19319978,
          0.18029557],
         ...,
         [0.4788336 , 0.4589783 , 0.497278